In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_credits.csv')
credits = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
movies.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [4]:
#to merge both cred and mov 
movies = movies.merge(credits,on='title')

In [5]:
#shape of combined movies is 23 coloums 
movies.shape

(4809, 23)

In [6]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']] #select only these coloums as they are needed only

In [7]:
movies.dropna(inplace=True) # to delete the movies that had null in any coloumn as it was 3 so we del the movies from db

In [8]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
#to check duplicate data ,0 in this case
movies.duplicated().sum()

0

In [10]:
movies.iloc[0].genres 

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [11]:
# now from the genre coloumn we have to make it from[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, 
#{"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# to ['action','adventure','fantasy','scifi'] 

In [12]:
import ast

In [13]:
def convert(text):
    L = []
    for i in ast.literal_eval(text): # used ast module and function literal_eval to convert string to list
        L.append(i['name']) 
    return L 

In [14]:
movies.dropna(inplace=True)

In [15]:
movies['genres'] = movies['genres'].apply(convert)

In [16]:
movies['keywords'] = movies['keywords'].apply(convert)


In [17]:
import ast
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [18]:
movies['cast'] = movies['cast'].apply(convert3)


In [19]:
import ast
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L
    

In [20]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [21]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [22]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [23]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [24]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [25]:
new_db = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [26]:
new_db['tags'] = new_db['tags'].apply(lambda x:" ".join(x))

In [27]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [28]:
import nltk #reffer page 3 and 4

In [29]:
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer()

In [30]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return  " ".join(y)

In [31]:
new_db['tags'] = new_db['tags'].apply(stem)

In [32]:
new_db['tags'] = new_db['tags'].apply(lambda x:x.lower()) #convert tags to lower case

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english') #page 3 of notes reffer and site for countervector

In [34]:
vector = cv.fit_transform(new_db['tags']).toarray() #as we recieved sparse vectors with shape (no of movies, 5000(words selected))

In [35]:
vector #obv not all movies will have those repeated words so it will be mostly 0

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
len(cv.get_feature_names_out()) # all the 5000 words

5000

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [38]:
similarity


array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [39]:
def recommend(movie):
    movie_index = new_db[new_db['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[movie_index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_db.iloc[i[0]].title)
    

In [40]:
recommend('The Dark Knight')

The Dark Knight Rises
Batman Begins
Batman Returns
Batman Forever
Batman


In [42]:
import pickle

In [44]:
pickle.dump(new_db.to_dict(),open('movies_dictionary.pkl','wb')) #opening in write binary mode so wb

In [45]:
pickle.dump(similarity,open('similarity.pkl','wb'))